In [1]:
import numpy as np
import pandas as pd

In [2]:
cd ml-100k/

/Users/yongli/A-Teaching/PDS/code/recsys/ml-100k


In [3]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=names)
df.head()

i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('u.item', sep='|', names=i_cols, encoding='latin-1')

items.head()

n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print(str(n_users) + ' users')
print(str(n_items) + ' items')

943 users
1682 items


In [4]:
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]

ratings_df = pd.DataFrame(ratings)
print(ratings_df.shape)
ratings_df.head()

(943, 1682)


,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
from sklearn.metrics.pairwise import cosine_similarity
userSimilarity = cosine_similarity(ratings, ratings)

data_ubs_fast = pd.DataFrame(userSimilarity)
print(data_ubs_fast.shape)
data_ubs_fast.head()

(943, 943)


,0,1,2,3,4,5,6,7,8,9,...,933,934,935,936,937,938,939,940,941,942
0,1.000000,0.166931,0.047460,0.064358,0.378475,0.430239,0.440367,0.319072,0.078138,0.376544,...,0.369527,0.119482,0.274876,0.189705,0.197326,0.118095,0.314072,0.148617,0.179508,0.398175
1,0.166931,1.000000,0.110591,0.178121,0.072979,0.245843,0.107328,0.103344,0.161048,0.159862,...,0.156986,0.307942,0.358789,0.424046,0.319889,0.228583,0.226790,0.161485,0.172268,0.105798
2,0.047460,0.110591,1.000000,0.344151,0.021245,0.072415,0.066137,0.083060,0.061040,0.065151,...,0.031875,0.042753,0.163829,0.069038,0.124245,0.026271,0.161890,0.101243,0.133416,0.026556
3,0.064358,0.178121,0.344151,1.000000,0.031804,0.068044,0.091230,0.188060,0.101284,0.060859,...,0.052107,0.036784,0.133115,0.193471,0.146058,0.030138,0.196858,0.152041,0.170086,0.058752
4,0.378475,0.072979,0.021245,0.031804,1.000000,0.237286,0.373600,0.248930,0.056847,0.201427,...,0.338794,0.080580,0.094924,0.079779,0.148607,0.071459,0.239955,0.139595,0.152497,0.313941


In [6]:
data_pre = pd.DataFrame(index=range(0,n_users),columns=items['movie title'])

epsilon=1e-9

for i in range(0,3): #len(data_pre.index)):
    for j in range(0,len(data_pre.columns)):
        if ratings_df.iloc[i][j] > 0:
            data_pre.iloc[i][j] = 0
        else:
            mask_rated_users = ratings_df[j] > 0
            top_neighbours = data_ubs_fast.loc[mask_rated_users,i].sort_values(ascending=False)[0:10].index
            top_neighbours_sim = data_ubs_fast.loc[mask_rated_users,i].sort_values(ascending=False)[0:10].values
            neighbours_ratings = ratings_df.iloc[top_neighbours,j]
            
            data_pre.iloc[i][j] = sum(neighbours_ratings*top_neighbours_sim)/(sum(top_neighbours_sim) + epsilon)

print(data_pre.shape)
data_pre.head(3)

(943, 1682)


movie title,Toy Story (1995),GoldenEye (1995),Four Rooms (1995),Get Shorty (1995),Copycat (1995),Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),Twelve Monkeys (1995),Babe (1995),Dead Man Walking (1995),Richard III (1995),...,Mirage (1995),Mamma Roma (1962),"Sunchaser, The (1996)","War at Home, The (1996)",Sweet Nothing (1995),Mat' i syn (1997),B. Monkey (1998),Sliding Doors (1998),You So Crazy (1994),Scream of Stone (Schrei aus Stein) (1991)
0,0,0,0,0,0,0,0,0,0,0,...,3,4,3,2,3,1,3,2,3,3
1,0,2.93996,2.29475,3.38894,3.39835,3.47237,3.28955,4.10071,3.88842,0,...,3,4,3,2,3,1,3,2,3,3
2,4.31023,2.90073,3.36236,3.46123,3.36267,2.8879,3.6805,4.13489,4.14859,3.33583,...,3,4,3,2,3,1,3,2,3,3


In [7]:
data_recommend = pd.DataFrame(index=data_pre.index, columns=['1','2','3','4','5','6'])

for i in range(0,len(data_pre.index)):
    data_recommend.iloc[i,0:] = data_pre.iloc[i,:].sort_values(ascending=False)[0:6].index

print(data_recommend.shape)
data_recommend.head(3)

(943, 6)


,1,2,3,4,5,6
0,Prefontaine (1997),"Saint of Fort Washington, The (1993)",Santa with Muscles (1996),"Great Day in Harlem, A (1994)",Someone Else's America (1995),They Made Me a Criminal (1939)
1,Prefontaine (1997),Star Kid (1997),Santa with Muscles (1996),"Great Day in Harlem, A (1994)",Marlene Dietrich: Shadow and Light (1996),Entertaining Angels: The Dorothy Day Story (1996)
2,Star Kid (1997),Prefontaine (1997),Santa with Muscles (1996),"Great Day in Harlem, A (1994)",Entertaining Angels: The Dorothy Day Story (1996),"Saint of Fort Washington, The (1993)"
